# Introduction to Natural Language Processing 2 Lab04

## Introduction

Your company wants to sell a moderation API tackling toxic content on Twitter. They ask you to come up with a model which detect toxic tweets. You remember your NLP classes, and start looking for existing models or datasets, and find a collection of [academic Twitter dataset on HuggingFace hub](https://huggingface.co/datasets/tweet_eval). Especially, the `hate` and `offensive` datasets seem close to what you are looking for.

In [24]:
# Get the datasets from HuggingFace
from datasets import load_dataset

hate_dataset = load_dataset("tweet_eval", "hate")
offensive_dataset = load_dataset("tweet_eval", "offensive")

train_hate = hate_dataset["train"]
valid_hate = hate_dataset["validation"]
test_hate = hate_dataset["test"]
train_offensive = offensive_dataset["train"]
valid_offensive = offensive_dataset["validation"]
test_offensive = offensive_dataset["test"]

Found cached dataset tweet_eval (/home/pili/.cache/huggingface/datasets/tweet_eval/hate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
100%|██████████| 3/3 [00:00<00:00, 1293.07it/s]
Found cached dataset tweet_eval (/home/pili/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
100%|██████████| 3/3 [00:00<00:00, 1778.50it/s]


### 1. (1 point) Pick one of the datasets between `hate` and `offensive`, and justify your choice. Remember that it is for a commercial application (there is a good and a bad answer).

The model needs to detect toxic tweets.

In [25]:
from IPython.display import Markdown, display

# Get a preview of the data
display(Markdown("### Hate dataset"))
display(Markdown("#### Label 0"))
display(train_hate.filter(lambda example: example["label"] == 0).select(range(5))["text"])
display(Markdown("#### Label 1"))
display(train_hate.filter(lambda example: example["label"] == 1).select(range(5))["text"])
display(Markdown("### Offensive dataset"))
display(Markdown("#### Label 0"))
display(train_offensive.filter(lambda example: example["label"] == 0).select(range(5))["text"])
display(Markdown("#### Label 1"))
display(train_offensive.filter(lambda example: example["label"] == 1).select(range(5))["text"])

### Hate dataset

#### Label 0

Loading cached processed dataset at /home/pili/.cache/huggingface/datasets/tweet_eval/hate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-f4b9014e43714767.arrow


['@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you…',
 'Hysterical woman like @user',
 'Me flirting- So tell me about your father...',
 'The Philippine Catholic bishops\' work for migrant workers should focus on families who are "paying the great...',
 'When cuffin season is finally over']

#### Label 1

...

Loading cached processed dataset at /home/pili/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-eb9ecfc3e8c05dd5.arrow


['@user Bono... who cares. Soon people will understand that they gain nothing from following a phony celebrity. Become a Leader of your people instead or help and support your fellow countrymen.',
 '@user Get him some line help. He is gonna be just fine. As the game went on you could see him progressing more with his reads. He brought what has been missing. The deep ball presence. Now he just needs a little more time',
 '@user @user She is great. Hi Fiona!',
 '@user @user @user @user @user @user @user @user @user @user @user @user @user @user @user This is the VetsResistSquadron"" is Bullshit.. They are girl scout veterans, I have never met any other veterans or served with anyone that was a gun control advocate? Have you?""',
 '@user @user Lol. Except he’s the most successful president in our lifetimes. He’s undone most of the damage Obummer did and set America on the right path again. #MAGA']

#### Label 1

Loading cached processed dataset at /home/pili/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343/cache-444e3b49659f88c2.arrow


['@user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president.',
 "@user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too  Who know with Liberals  They're all optics.  No substance",
 '@user Your looking more like a plant #maga #walkaway',
 '@user Antifa would burn a Conservatives house down and CNN would be there lighting the torches &amp; throwing gas on the flames.',
 '@user They cite Jones being banned for violating Twitter\'s ToS. There are blue checkmarks spewing the same, if not worse, kind of shit. If you are going to play the anyone can get banned"" card. Shouldn\'t these people also receive bans and suspensions? #VerifiedHate""']

From the preview of the two datasets we can see that the `hate` dataset tackles more toxic messages than the `offensive` one.  


In [26]:
chosen = hate_dataset
train = chosen["train"]
valid = chosen["validation"]
test = chosen["test"]

## Evaluating the dataset

Let's start with a data analysis.  
### 1. (1 point) Describe the dataset. Look at the splits, proportion of classes, and see what you can figure out by just looking at the text.

In [27]:
import pandas as pd

# Get proportion of splits and classes
print("Training split:", len(train), "\n" + str(pd.Series(train["label"]).value_counts()),
      "\nValidation split:", len(valid), "\n" + str(pd.Series(valid["label"]).value_counts()),
        "\nTest split:", len(test), "\n" + str(pd.Series(test["label"]).value_counts()))

Training split: 9000 
0    5217
1    3783
dtype: int64 
Validation split: 1000 
0    573
1    427
dtype: int64 
Test split: 2970 
0    1718
1    1252
dtype: int64


From the proportion of classes we can see that there are less negative (label 1) than positive tweets.

### 2. (3 points) Use BERTopic to extract the topics within the data, and the main topics within each class. Please, think about fixing the random seed


#### with the raw data

In [28]:
import bertopic
from umap import UMAP

# fix the seed for reproducibility
umap_model = UMAP(random_state=42)

# Use BERTopic model
# we use all-MiniLM-L6-v2
topic_model = bertopic.BERTopic(language="english", calculate_probabilities=True, verbose=True, embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)
topics, probs = topic_model.fit_transform(train["text"])


Batches: 100%|██████████| 282/282 [00:03<00:00, 83.02it/s] 
2023-06-10 14:19:43,983 - BERTopic - Transformed documents to Embeddings
2023-06-10 14:19:52,523 - BERTopic - Reduced dimensionality
2023-06-10 14:19:55,447 - BERTopic - Clustered reduced embeddings


In [29]:
# get topic per class
topics_per_class = topic_model.topics_per_class(train["text"], train["label"])

topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)

2it [00:00, 11.50it/s]


#### with balanced data

In [30]:
import random
from datasets import Dataset
random.seed(42)

# Separate the samples based on their labels
label_0_samples = [sample for sample in train if sample["label"] == 0]
label_1_samples = [sample for sample in train if sample["label"] == 1]

# Randomly select 1000 samples from each label
balanced_train = random.sample(label_0_samples, 3000) + random.sample(label_1_samples, 3000)

# Shuffle the order of the balanced train set
random.shuffle(balanced_train)
balanced_train_dataset = Dataset.from_dict({"text": [sample["text"] for sample in balanced_train], "label": [sample["label"] for sample in balanced_train]})
print("Balanced training set:", len(balanced_train_dataset), "\n" + str(pd.Series(balanced_train_dataset["label"]).value_counts()))

Balanced training set: 6000 
0    3000
1    3000
dtype: int64


In [31]:
# fix the seed for reproducibility
umap_model = UMAP(random_state=42)

# Use BERTopic model
# we use all-MiniLM-L6-v2
topic_model = bertopic.BERTopic(language="english", calculate_probabilities=True, verbose=True, embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)
topics, probs = topic_model.fit_transform(balanced_train_dataset["text"])

Batches: 100%|██████████| 188/188 [00:02<00:00, 81.13it/s] 
2023-06-10 14:19:58,932 - BERTopic - Transformed documents to Embeddings
2023-06-10 14:20:04,524 - BERTopic - Reduced dimensionality
2023-06-10 14:20:05,111 - BERTopic - Clustered reduced embeddings


In [32]:
# get topic per class
topics_per_class = topic_model.topics_per_class(balanced_train_dataset["text"], balanced_train_dataset["label"])

topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)

2it [00:00, 20.60it/s]


### 3. (1 point) What do you think about the results? How do you think it could impact a model trained on these data?

We printed a second graph with only 3000 samples per class to see if it has an impact on the frequency (and it does).

We mainly see that a main topic are differents between classes :

between label 1 and 0 we have some topics mainly represented in label 1 (hatefull) such as : `men_not_all_women`, `cunt_bitch_whore_user` or `builthawall` 
on the other side we have : `when_you_oure` or `refugee_welcome` mainly represented in class 0  
but most of the topics are represented in boths so it will be difficult to detect offensive content  

these information may be used to detect hatefull content


## Evaluate a model

### (2 points) Evaluate their model on the test split of the dataset you picked, using precision, recall, and F1-score.

In [33]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='hate'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [79]:
import torch
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tqdm

# Evaluate the model on the test split of the hate dataset, using precision, recall, and F1-score.
def evaluate_model(model, dataset):
    # Get the predictions
    predictions = []
    scores_list = []
    for i in tqdm.tqdm(range(len(dataset))):
        text = dataset[i]["text"]
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        predicted_label = np.argmax(scores)
        predictions.append(predicted_label)
        scores_list.append(scores)

    # Calculate the accuracy
    accuracy = accuracy_score(dataset["label"], predictions)

    # Calculate the precision, recall, and F1-score
    precision = precision_score(dataset["label"], predictions)
    recall = recall_score(dataset["label"], predictions)
    f1 = f1_score(dataset["label"], predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    return predictions, scores_list


predictions, scores_list = evaluate_model(model, test)

100%|██████████| 2970/2970 [04:23<00:00, 11.27it/s]

Accuracy: 0.5768
Precision: 0.4989
Recall: 0.9481
F1-score: 0.6538


We have an accuracy of 0.5768  
a precision of 0.4989  
a recall of 0.9481
and a F1 score of 0.6538  

the model has a high recall meaning its good at finding hatefull content although it's precision is a bit less good

### (2 points) Look for prediction failures. Extract the top 5 misclassified tweets (highest score in wrong class) for each class and discuss what could be wrong with the model.

In [101]:
# extract the 5 most failed predictions
wrong_predictions = []
predictions_scores_diff = []
idx_list = []
for i in tqdm.tqdm(range(len(predictions))):
    if predictions[i] != test["label"][i]:
        # [score_not_hate, score_hate]
        score_diff = scores_list[i][test["label"][i]-1]

        # take the top 5 most failed predictions
        if len(wrong_predictions) < 5:
            wrong_predictions.append((test["text"][i], predictions[i], test["label"][i]))
            predictions_scores_diff.append(score_diff)
            idx_list.append(i)

        # replace the worst prediction with the current one
        else:
            worst_prediction_index = np.argmin(predictions_scores_diff)
            if score_diff > predictions_scores_diff[worst_prediction_index]:
                wrong_predictions[worst_prediction_index] = (test["text"][i], predictions[i], test["label"][i])
                predictions_scores_diff[worst_prediction_index] = score_diff
                idx_list[worst_prediction_index] = i

100%|██████████| 2970/2970 [00:02<00:00, 996.11it/s] 


In [102]:
# print 5 predictions false
for i in range(len(wrong_predictions)):
    display(Markdown("#### Prediction " + str(i)))
    display(Markdown("##### Text"))
    display(wrong_predictions[i][0])
    display(Markdown("##### Predicted label"))
    display(labels[wrong_predictions[i][1]])
    display(Markdown("##### Actual label"))
    display(labels[wrong_predictions[i][2]])
    display(Markdown("##### Scores"))
    display('Not hate: ' + str(scores_list[idx_list[i]][0]) + ' Hate: ' + str(scores_list[idx_list[i]][1]))
    display(Markdown("##### Difference"))
    display(predictions_scores_diff[i])
    print()

#### Prediction 0

##### Text

'#BuildTheWall National Security @user @user @user @user @user @user @user @user @user @user #NoDACAdeal #NoAmnesty #MandateEverify #EndChainMigration #MAGA #MakeDCListen'

##### Predicted label

'hate'

...

'not-hate'

##### Scores

'Not hate: 0.012590758 Hate: 0.98740923'

##### Difference

0.98740923

The model has problem samll tweets using specific hastags, mainly with the ones talking of subject such as the immigration and the wall of donald Trump.  
It also has a lot of problem with tweets with a lot of @user (meaning it's a conversation with a lot of people) maybe the algo would be better with more context

### (2 points) Extract the top 10 tweets your model is most confident about in the target class (offensive or hateful), the top 10 in the neutral class, and the top 10 your model is most uncertain about. Do you believe the model is doing a great job?

In [136]:
# extract the 10 most successful predictions
correct_predictions = []
predictions_scores_diff = []
idx_list = []
for i in tqdm.tqdm(range(len(predictions))):
    if predictions[i] == 1:
        # [score_not_hate, score_hate]
        score_diff = scores_list[i][test["label"][i]-1]

        # take the top 10 most successful predictions
        if len(correct_predictions) < 10:
            correct_predictions.append((test["text"][i], predictions[i], test["label"][i]))
            predictions_scores_diff.append(score_diff)
            idx_list.append(i)

        # replace the worst prediction with the current one
        else:
            best_prediction_index = np.argmin(predictions_scores_diff)
            if score_diff < predictions_scores_diff[best_prediction_index]:
                correct_predictions[best_prediction_index] = (test["text"][i], predictions[i], test["label"][i])
                predictions_scores_diff[best_prediction_index] = score_diff
                idx_list[best_prediction_index] = i


100%|██████████| 2970/2970 [00:01<00:00, 1737.46it/s]


In [137]:
# extract the 10 worst predictions but still correct
correct_predictions_worst = []
predictions_scores_diff_worst = []
idx_list_worst = []
for i in tqdm.tqdm(range(len(predictions))):
    if predictions[i] == 1:
        # [score_not_hate, score_hate]
        score_diff = scores_list[i][test["label"][i]-1]

        # take the top 10 worst successful predictions
        if len(correct_predictions_worst) < 10:
            correct_predictions_worst.append((test["text"][i], predictions[i], test["label"][i]))
            predictions_scores_diff_worst.append(score_diff)
            idx_list_worst.append(i)

        # replace the worst prediction with the current one
        else:
            best_prediction_index = np.argmax(predictions_scores_diff_worst)
            if score_diff > predictions_scores_diff_worst[best_prediction_index]:
                correct_predictions_worst[best_prediction_index] = (test["text"][i], predictions[i], test["label"][i])
                predictions_scores_diff_worst[best_prediction_index] = score_diff
                idx_list_worst[best_prediction_index] = i

100%|██████████| 2970/2970 [00:01<00:00, 1778.72it/s]


In [138]:
# print 10 random predictions correct
correct_predictions_random = []
predictions_scores_diff_random = []
idx_list_random = []
for i in tqdm.tqdm(range(len(predictions))):
    if predictions[i] == 1:
        # [score_not_hate, score_hate]
        score_diff = scores_list[i][test["label"][i]-1]

        # take the top 5 worst successful predictions
        if len(correct_predictions_random) < 10:
            correct_predictions_random.append((test["text"][i], predictions[i], test["label"][i]))
            predictions_scores_diff_random.append(score_diff)
            idx_list_random.append(i)
        else:
            break
            

  0%|          | 12/2970 [00:00<00:13, 214.33it/s]


In [139]:
# print 10 predictions correct
display(Markdown("## Target Hate"))

display(Markdown("### best Correct predictions"))
for i in range(len(correct_predictions)):
    display(Markdown(correct_predictions[i][0]))

# print 10 predictions correct but worst
display(Markdown("### worst Correct predictions"))
for i in range(len(correct_predictions_worst)):
    display(Markdown(correct_predictions_worst[i][0]))

# print 10 neutral predictions correct
display(Markdown("### neutral Correct predictions"))
for i in range(len(correct_predictions_random)):
    display(Markdown(correct_predictions_random[i][0]))

## Target Hate

### best Correct predictions

@user , you are correct that Reid certainly is a weasel. Sadly, we've got our own weasels; @user Sen McConnell & @user .The corrupt Mueller investigation w/be STOPPED if those 3 did their jobs.#MAGA #KAG #POTUS #Trump #NEWS #VoteRed #NoDACA #USA

@user @user Those People Invaded Us!!! They DO NOT BELING HERE & HAVE NO RIGHTS! Its #AmericaFIRST! Open Your House To Them If Your That IGNORANT! & Yes Im A #Christian Too! #NODACA!

stop JUDGING bitches by there cover, jus cuz she bad don't mean she's a catch shawdy could be a whore 👀 das opposite of a keeper

...

@user @user Muslims attacked US on 9/11, 3000 killed. Subsequently we allowed more Muslims in our country?!?Does this make any sense to you?You can thank Obama, his liberals minions, RINO's & political correctness. #MuslimBan #BanIslam #TravelBan #DrainTheDeepState #DeportThemAll

@user @user Like he ever kept out any threats. He's lying as usual. #BuildThatWall

#germany deserves to called #cuck/land they allow in #isis fighters as #Refugees even if they raped underaged girls. this #yazidi girl is scared of this and she is forced to see the face of her rapist.#refugeesnotwelcome should apply to these scum

@user @user They can scrim whoever they fucking want this isn't a fucking chall you dumb bitch

Ok bitch nigga. You don't even watch basketball! Stfu. Who was the laker coach this year? Bet yo hoe ass look it up…

The model seems quite good although it's still hard to correctly visualize the data

### (2 points) What are the advantages of using a pre-trained transformer vs naive Bayes?

Using a pre-trained transformer model, such as BERT, offers several advantages over a naive Bayes classifier. These include the ability to learn complex patterns, transfer learning capabilities, superior language understanding, handling long-range dependencies, and continuous learning.  
However, transformers like BERT are computationally expensive and less interpretable compared to naive Bayes classifiers.

## Annontate Data

### (1 point) Extract about 100 tweets containing at least 20% of your target class (offensive/hateful), from the 10K tweets provided. You can use the pretrained model to help you find tweets in the target class.

In [141]:
# Extract 100 tweets from the test set that contain at least 20% of hate score
hate_tweets = []
hate_tweets_scores = []
hate_tweets_idx = []
for i in tqdm.tqdm(range(len(predictions))):
    if scores_list[i][1] > 0.2:
        hate_tweets.append(test["text"][i])
        hate_tweets_scores.append(scores_list[i][1])
        hate_tweets_idx.append(i)

# print 100 tweets that contain at least 20% of hate score
display(Markdown("### 5 of the 100 tweets that contain at least 20% of hate score"))
for i in range(5):
    display(Markdown(hate_tweets[i]))

100%|██████████| 2970/2970 [00:04<00:00, 689.94it/s]


### 5 of the 100 tweets that contain at least 20% of hate score

@user , you are correct that Reid certainly is a weasel. Sadly, we've got our own weasels; @user Sen McConnell & @user .The corrupt Mueller investigation w/be STOPPED if those 3 did their jobs.#MAGA #KAG #POTUS #Trump #NEWS #VoteRed #NoDACA #USA

Whoever just unfollowed me you a bitch

@user @user Those People Invaded Us!!! They DO NOT BELING HERE & HAVE NO RIGHTS! Its #AmericaFIRST! Open Your House To Them If Your That IGNORANT! & Yes Im A #Christian Too! #NODACA!

stop JUDGING bitches by there cover, jus cuz she bad don't mean she's a catch shawdy could be a whore 👀 das opposite of a keeper

how about i knock heads off and send them gift wrapped to your moms house you dumb raggedy bird bitch ass hoes

### (3 points) Altogether, write down an annotation guildeline (which should be at least 2/3 of a page long).